## Ages

Rewriting the code for "Benchmark maps of 33 years of secondary forest age for Brazil" to run with Collection 8 and include only class 3.

In [ ]:
# making masks
# Anthropic, urban and Water Mask

LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9]
# INDEX ## 3 = forest
# 15 = pasture
# 39 = soy
# 20 = sugar cane
# 40 = rice
# 62 = cotton
# 41 = other temporary crop
# 46 = coffee
# 47 = citrus
# 35 = palm oil
# 48 = other perennial crop
# 9 = forest plantation
lulc_palette = ["#f1c232", "#FFFFB2", "#FFD966", "#E974ED", "#D5A6BD", "#e075ad", "#C27BA0", "#982c9e", "#e787f8", "#cd49e4", "#ad4413"]

empty = ee.Image().byte()

for i in range(first_year, last_year + 1):
    year = 'classification_' + str(i)
    anthropic = lulc.select(year).remap(LU_index, [1] * len(LU_index), 0).rename(year)
    empty = empty.addBands(anthropic)

anthropic_mask = empty.select(empty.bandNames().slice(1)) # we only want areas that DO show anthropic activity this year

# Replace 'YOUR_WATER_IMAGE_ID' with the actual water image ID you are working with
w_mask  = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("max_extent").clip(roi).remap([0,1],[1,0]);

urban = ee.Image("DLR/WSF/WSF2015/v1").clip(roi)
inverse_mask = urban.eq(1).Not()  # This will invert the mask, city pixels will be False (or 0) and non-urban pixels will be True (or 1)
urban = urban.updateMask(inverse_mask)  # This will replace 1 values (city pixels) with NA
urban_mask = urban.unmask(1)  # This will replace NA values (non-urban pixels) with 1

# Define a color palette
# palette = ['blue', 'red']  # Blue for 0, Red for 1
# vizpar2 = {'min': 1, 'max': len(LU_index), 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Add the layer to the map with the color palette
# Map.addLayer(w_mask, {'min': 0, 'max': 1, 'palette': palette}, 'w_mask')
# Map.addLayer(urban_mask, {'palette': palette}, 'urban_mask')
# Map.addLayer(anthropic_mask.select('classification_2020'), vizpar2, "anthropic")
# Map

In [ ]:
# 1. Reclassifying MapBiomas Data # Step 1
empty = ee.Image().byte();

for i in range(first_year, last_year+1):
    year = 'classification_' + str(i)
    forest = lulc.select(year)
    forest = forest.remap([3,6], [1,1], 0) # Forest Formation and Flooded Forest classes from MapBiomas Project
    empty = empty.addBands(forest.rename(ee.String(year)))

mapbiomas_forest = empty.select(empty.bandNames().slice(1))

In [ ]:
# 2. Mapping the Annual Increment of Secondary Forests # Step 2
regro = ee.Image().byte()
defor = ee.Image().byte()

for i in range(first_year, last_year):  # 1986-2020
    year1 = f'classification_{i}'
    year2 = f'classification_{i + 1}'
    a_mask = anthropic_mask.select(year1);
    forest1 = mapbiomas_forest.select(year1).remap([0, 1], [0, 2])  # Change forest pixels in 1985 to 2 years old
    forest2 = mapbiomas_forest.select(year2)
    # addition is 0 if was nonforest before and after; 1 if it was gained; 2 if it was forest before and then was lost; 3 if it was forest in both.
    sforest = forest1.add(forest2).multiply(a_mask).multiply(w_mask).multiply(urban_mask)
    for_gain = sforest.remap([0, 1, 2, 3], [0, 1, 0, 0]).rename(year2)
    for_loss = sforest.remap([0, 1, 2, 3], [0, 0, 1, 0]).rename(year2)
    regro = regro.addBands(for_gain)
    defor = defor.addBands(for_loss)

regro = regro.select(regro.bandNames().slice(1))  # Shows all years in which forest was gained.
# here, we could just mask by pixels that are forest in 2020 and find the year of last gain.

In [ ]:
# 3. Mapping the Annual Extent of Secondary Forests # Step 3
extent = ee.Image().byte()
# add pixels that gained forest in 1986
extent = extent.addBands(regro.select('classification_1986').rename('classification_1986'))

for i in range(first_year + 1, last_year): #1987 to 2020
    year = f'classification_{i}' #1986
    year2 = f'classification_{i + 1}' #1987
    for_gain = regro.select(year2)
    acm_forest = extent.select(year).add(for_gain) #pixels that gained forest in 1986 + pixels that gained forest in 1987
    old_values = list(range(37))
    new_values = [0, 1] + [1] * 35
    remap = acm_forest.remap(old_values, new_values)
    # mask (multiply) by pixels that were shown to be forest in 1987, hence eliminating any that may have regrown in 1986 but lost cover in 1987
    extent = extent.addBands(remap.multiply(mapbiomas_forest.select(year2)).rename(year2))

extent = extent.select(extent.bandNames().slice(1))

In [ ]:
# 4. Calculating and Mapping the Age of Secondary Forests # Step 4
ages = ee.Image().byte()
ages = ages.addBands(extent.select('classification_1986').rename('classification_1986'))
ages = ages.slice(1) # remove "constant" band
age_total = ages # will use this as the "last total age" to keep iteratively adding values

for i in range(first_year + 1, last_year):
    year = f'classification_{i + 1}'# 1987-2020
    sforest = extent.select(year) # forest cover in 1987
    age_total = age_total.add(sforest) # 1 year old forests in 1986 + cover in 1987
    f_year = mapbiomas_forest.select(year)
    age_total = age_total.multiply(f_year) # mask by pixels that were forest that year, removing any forest loss
    ages = ages.addBands(age_total.rename(year))

ages = ages.updateMask(ages) #keep only values as ages or NA

#ages range from 1 for those regrown in 2019-2020 to 35 for those regrown in 1985-1986
age = ages.select('classification_2020').rename('age')

# vizpar = {'min': 1, 'max': last_year - first_year, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(age, vizpar, "ages")
# Map